# Fitting a Galprop Model to the Full Sky. 

We reproduce the global fitting analysis of http://arxiv.org/abs/1603.06584.  The fit is done in three stages using output files from Galprop.  First we fit the high latitude sky, then we fit the outer galaxy, and finally, the inner Galaxy.  XCO is allowed to float freely in this case.  

The output format is a single HDF5 file, which will be useful in later analysis. If one does not wish to fit a galprop model, but instead just use the output, see the tutorial, using Galprop output. 


This analysis assumes that the galprop output has 9 rings for each gas component.  These input gas maps to galprop can be found in the support_files folder of the GammaLike github repo.

In [1]:
import sys
sys.path.append('/data/GammaLike/')
from GammaLike import Analysis 

basepath = '/data/GammaLike/testing/'
tag = 'P8R2_CLEAN_V6_calore' 
A = Analysis.Load(basepath+tag+'.GLanalysis')

support_files_path = '/data/GammaLike/support_files/'


A.AddIsotropicTemplate(isofile=support_files_path+'/IGRB_ackerman_2014_modA.dat',
                       fixNorm=False, fixSpectrum=True) 

A.AddPointSourceTemplate(fixNorm=True, # Fix the template normalization in all bins.
                         # This was generated in "Starting a New Analysis"  tutorial
                         pscmap=A.basepath + '/PSC_' + A.tag + '_fgl3_with_ext.npy') 

A.CalculatePixelWeights(diffuse_model=A.basepath+'/'+'fermi_diffuse_'+A.tag+'.npy',
                        psc_model=A.basepath + '/PSC_' + A.tag + '_fgl3_with_ext.npy',
                        alpha_psc=5., f_psc=0.1) # see 1409.0042 Eq.(2.6)

A.AddFermiBubbleTemplate(template_file=support_files_path+'/new_bubble_template.fits.gz',
                         spec_file=support_files_path+'/reduced_bubble_spec_apj_793_64.dat', fixSpectrum=True, fixNorm=False)
# Load the binned photon file. 
A.BinPhotons(infile='binned_photons_'+A.tag+'.npy')

# Loading a Galprop Model 

Now let us load a Galprop model.  Currently, the GlobalFitter requires that the gas map used in galprop has 9 rings.  These gas maps can be found in the github repo support files. The individual components must be output by Galprop as well by setting the Galdef keyword "gamma_rays = 2".   For custom analyses, this source code can be modified relatively easily.  In our case we have a model called "Mod_A_2D" which corresponds to Mod A from Calore 2015 (http://arxiv.org/pdf/1409.0042v1.pdf).  This takes a bit to run since the 25 or so Galprop output files need to be decompressed.

In [2]:
from GammaLike import GlobalFitter

fit = GlobalFitter.GlobalFitter()
fit.GenDiffuse(A, # Our analysis above
               basedir='/data/galprop2/output', # The directory containing galprop's output
               tag='Mod_A_2D', # The "tag" for the galprop model which is the part between "v54" and ".gz" in the output
               fixSpectrum=True,  # When fitting the XCO rings, the spectrum is fixed to that of galprop
               fix_xco=False # We want to fit the XCO rings! 
              )



Adding HI/HII ring 1
Adding HI/HII ring 2
Adding HI/HII ring 3
Adding HI/HII ring 4
Adding HI/HII ring 5
Adding HI/HII ring 6
Adding HI/HII ring 7
Adding HI/HII ring 8
Adding HI/HII ring 9
Adding H2 ring 1
Adding H2 ring 2
Adding H2 ring 3
Adding H2 ring 4
Adding H2 ring 5
Adding H2 ring 6
Adding H2 ring 7
Adding H2 ring 8
Adding H2 ring 9
Adding ics 1
Adding ics 2
Adding ics 3


/data/GammaLike/GammaLike/Tools.py:399: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  master = np.zeros(12*nside**2)


Adding Templates to stack


<GammaLike.Analysis.Analysis instance at 0x7f3636ebdcb0>

The analysis A now has all of the required templates added and we can go ahead with the fit.  Note that this is analyzing the entire sky so it takes 20-30 minutes to run at 1/4 degree resolution. We refer the reader to http://arxiv.org/abs/1603.06584 and the source code for more information on how the fit is performed. 

In [3]:
A.PrintTemplates()

                NAME                    LIMITS      VALUE    FIXNORM    FIXSPEC   SRCCLASS
            pi0HIHII              [None, None]  1.000e+00      False       True        GEN
                 PSC              [None, None]  1.000e+00       True       True        PSC
            pi0_H2_1              [None, None]  1.000e+00      False       True        GEN
                 ics              [None, None]  1.000e+00      False       True        GEN
            pi0_H2_3              [None, None]  1.000e+00      False       True        GEN
            pi0_H2_2              [None, None]  1.000e+00      False       True        GEN
            pi0_H2_5              [None, None]  1.000e+00      False       True        GEN
           Isotropic              [None, None]  1.000e+00      False       True        ISO
            pi0_H2_7              [None, None]  1.000e+00      False       True        GEN
            pi0_H2_6              [None, None]  1.000e+00      False       True        GEN

In [4]:
# fit.RunFit() returns an iMinuit object containing the fit results, the -log-likelihood of the fit, and a residual skymap
m, fval, res = fit.RunFit(A)

Running Local Ring Fit...
Masking completed in 9.9041e-01 s
Code generation completed in 2.2021e-02 s
Fallback to CPU mode.  (Failed to import cudamat libraries.)
ncall/-LL 105 -508165.403691

FCN = -507761.77453 
 TOTAL NCALL = 109 
 NCALLS = 109 
 
 
 EDM = 0.00960012532729 
 GOAL EDM = 0.05 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,pi0HIHII,7.209173e-01,6.549522e-04,0.000000e+00,0.000000e+00,0.0,,
2,PSC,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
3,pi0_H2_1,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
4,ics,4.251261e-01,9.319030e-04,0.000000e+00,0.000000e+00,0.0,,
5,pi0_H2_3,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
6,pi0_H2_2,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
7,pi0_H2_5,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
8,Isotropic,1.683269e+00,2.393598e-03,0.000000e+00,0.000000e+00,0.0,,
9,pi0_H2_7,4.646647e-01,1.427283e-03,0.000000e+00,0.000000e+00,0.0,,
10,pi0_H2_6,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED


 Migrad completed fitting   1.55e+02 s
isotropic value: 1.68326906807
X_CO adjustment (this is not XCO value, it is multiplier for galdef values of xco): [ 1.          1.          1.          1.          1.          1.
  0.46466474  1.          1.        ]
Running Outer Rings Fit...
Masking completed in 9.9103e-02 s
Code generation completed in 6.9849e-03 s
Fallback to CPU mode.  (Failed to import cudamat libraries.)
ncall/-LL 115 -10709959.3439

FCN = -10709561.5541 
 TOTAL NCALL = 120 
 NCALLS = 120 
 
 
 EDM = 0.0035196947093 
 GOAL EDM = 0.05 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,pi0HIHII,8.412198e-01,1.078995e-03,0.000000e+00,0.000000e+00,0.0,,
2,PSC,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
3,pi0_H2_1,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
4,ics,6.231192e-01,3.332756e-03,0.000000e+00,0.000000e+00,0.0,,
5,pi0_H2_3,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
6,pi0_H2_2,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
7,pi0_H2_5,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
8,Isotropic,1.683269e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,,FIXED
9,pi0_H2_7,4.646647e-01,1.000000e-01,0.000000e+00,0.000000e+00,0.0,,FIXED
10,pi0_H2_6,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED


 Migrad completed fitting   1.13e+01 s
X_CO adjustment (this is not XCO value, it is multiplier for galdef values of xco): [  1.           1.           1.           1.           1.           1.
   0.46466474   0.70563032  41.49037271]
Running Inner Rings Fit...
Masking completed in 7.3913e-02 s
Code generation completed in 2.1054e-02 s
Fallback to CPU mode.  (Failed to import cudamat libraries.)
ncall/-LL 340 -26110427.754

FCN = -26110039.7153 
 TOTAL NCALL = 341 
 NCALLS = 341 
 
 
 EDM = 0.0636493855179 
 GOAL EDM = 0.25 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,pi0HIHII,6.314298e-01,1.039674e-03,0.000000e+00,0.000000e+00,0.0,,
2,PSC,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,200.0,FIXED
3,pi0_H2_1,1.360426e+00,1.406408e-02,0.000000e+00,0.000000e+00,0.0,200.0,
4,ics,7.300975e-01,1.579455e-03,0.000000e+00,0.000000e+00,0.0,,
5,pi0_H2_3,2.600107e+00,2.880365e-02,0.000000e+00,0.000000e+00,0.0,200.0,
6,pi0_H2_2,2.947190e+00,5.558209e-02,0.000000e+00,0.000000e+00,0.0,200.0,
7,pi0_H2_5,1.781694e+00,6.779166e-03,0.000000e+00,0.000000e+00,0.0,200.0,
8,Isotropic,1.683269e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.0,,FIXED
9,pi0_H2_7,4.646647e-01,1.000000e-01,0.000000e+00,0.000000e+00,0.0,,FIXED
10,pi0_H2_6,9.847280e-01,4.512229e-03,0.000000e+00,0.000000e+00,0.0,200.0,


 Migrad completed fitting   2.55e+01 s
X_CO adjustment (this is not XCO value, it is multiplier for galdef values of xco): [  1.36042623   2.94718973   2.60010692   2.23842372   1.78169375
   0.98472797   0.46466474   0.70563032  41.49037271]


In [7]:
reload(GlobalFitter)

fit.WriteHDF5(fname='/data/GammaLike/testing/Mod_A_2D.hdf5',  # Filename of the output HDF5 file 
              basedir='/data/galprop2/output/',  # Galprop output folder
              tag='Mod_A_2D', # 
              m=m) #  iMinuit optimizer object. 
fit.AddMetadata(fname='/data/GammaLike/testing/Mod_A_2D.hdf5',  # Filename of the output HDF5 file 
                basedir='/data/galprop2/GALDEF', # Galdef folder which will be used to add metadata to the output HDF5 file. 
                tag='Mod_A_2D', 
                fval=fval, # fval contains the -log-likelihood from the local, outer, and inner ROIs respectively. 
                A=A, m=m)

(33, 786432)
Adding HI/HII ring 1
Adding HI/HII ring 2
Adding HI/HII ring 3
Adding HI/HII ring 4
Adding HI/HII ring 5
Adding HI/HII ring 6
Adding HI/HII ring 7
Adding HI/HII ring 8
Adding HI/HII ring 9
Adding ICS
Closed HDF5 file.
Adding fit metadata


TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [5]:
print A.evclass, type(A.evclass)
print A.convtype, type(A.convtype)

128 <type 'int'>
None <type 'NoneType'>
